In [6]:
import pandas as pd
import altair as alt
import numpy as np
import sqlite3

# Create DataBase and Add Table

In [7]:
conn = sqlite3.connect('Netflix.db')

cur = conn.cursor()

# cur.execute("""CREATE TABLE NFLX (
#     date string,
#     low float,
#     open float,
#     volume integer,
#     high float,
#     close float
# )""")

# Fill Table

In [8]:
path = 'C:/Users/Blake Dennett/Downloads/Spring2023/appliedProgramming/Data/stock_market_data/sp500/csv/j-p/nflx.csv'
df = pd.read_csv(path)
# iterate through each row of dataframe
for r in df.iterrows():
    row = r[1].tolist()
    to_insert = tuple(row[0:6])
    # add row into db
    # cur.execute(f"INSERT INTO NFLX VALUES {to_insert}")

conn.commit()

five_rows = pd.read_sql_query( 
    'SELECT * FROM NFLX LIMIT 5',
    conn)

five_rows

    

,date,low,open,volume,high,close
0,23-05-2002,1.145714,1.156429,104790000,1.242857,1.196429
1,24-05-2002,1.197143,1.214286,11104800,1.225000,1.210000
2,28-05-2002,1.157143,1.213571,6609400,1.232143,1.157143
3,29-05-2002,1.085714,1.164286,6757800,1.164286,1.103571
4,30-05-2002,1.071429,1.107857,10154200,1.107857,1.071429


# Deleting Duplicate Data

In [9]:
# cur.execute("DELETE FROM nflx WHERE date NOT IN (SELECT MIN(date) FROM nflx GROUP BY date);")

# Filtering Data

In [10]:
cur.execute("SELECT * FROM NFLX WHERE date = '23-05-2002'")
cur.fetchall()

[('23-05-2002',
  1.1457140445709229,
  1.1564290523529053,
  104790000,
  1.2428569793701172,
  1.1964290142059326)]

# Creating New Data Table

In [14]:
# cur.execute("""CREATE TABLE NFLX_ANALYSIS (
#             date string
# )""")

# dates = cur.execute("SELECT date FROM NFLX")

# query = "SELECT high, low FROM NFLX "

# for date in dates:
#     cur.execute(f"INSERT INTO NFLX_ANALYSIS VALUES ({date}, )")

('23-05-2002',)
('24-05-2002',)
('28-05-2002',)
('29-05-2002',)
('30-05-2002',)
('31-05-2002',)
('03-06-2002',)
('04-06-2002',)
('05-06-2002',)
('06-06-2002',)
('07-06-2002',)
('10-06-2002',)
('11-06-2002',)
('12-06-2002',)
('13-06-2002',)
('14-06-2002',)
('17-06-2002',)
('18-06-2002',)
('19-06-2002',)
('20-06-2002',)
('21-06-2002',)
('24-06-2002',)
('25-06-2002',)
('26-06-2002',)
('27-06-2002',)
('28-06-2002',)
('01-07-2002',)
('02-07-2002',)
('03-07-2002',)
('05-07-2002',)
('08-07-2002',)
('09-07-2002',)
('10-07-2002',)
('11-07-2002',)
('12-07-2002',)
('15-07-2002',)
('16-07-2002',)
('17-07-2002',)
('18-07-2002',)
('19-07-2002',)
('22-07-2002',)
('23-07-2002',)
('24-07-2002',)
('25-07-2002',)
('26-07-2002',)
('29-07-2002',)
('30-07-2002',)
('31-07-2002',)
('01-08-2002',)
('02-08-2002',)
('05-08-2002',)
('06-08-2002',)
('07-08-2002',)
('08-08-2002',)
('09-08-2002',)
('12-08-2002',)
('13-08-2002',)
('14-08-2002',)
('15-08-2002',)
('16-08-2002',)
('19-08-2002',)
('20-08-2002',)
('21-08-